# Lecture 6: cross-validation and bootstrap

In [ ]:
import numpy as np
from ISLP import load_data
import statsmodels.api as sm

from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split

In [ ]:
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Naive validation set approach

In [ ]:
Auto = load_data('Auto')
print(Auto.shape)
# As there are 392 observations, we split into two equal sets of size 196
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

In [ ]:
X_train = # TODO: retrieve horsepower from Auto_train
X_train = # TODO: add constant to X_train
y_train = # TODO: retrieve 'mpg' from auto_train
model = # TODO: initialize OLS
results = # TODO: fit OLS
results.params

In [ ]:
results.summary()

In [ ]:
# TODO: similar to above steps, now on the validation set

X_valid = # TODO
X_valid = # TODO
y_valid = # TODO
valid_pred = # TODO
np.mean((y_valid - valid_pred)**2) # compute MSE on validation set

In [ ]:
# evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set
def evalMSE(train, test, d):
    X_train = train['horsepower']
    X_train = np.power.outer(X_train.values, np.arange(d+1))
    y_train = train['mpg']
    
    X_test = test['horsepower']
    X_test = np.power.outer(X_test.values, np.arange(d+1))
    y_test = test['mpg']
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [ ]:
# fit polynomials of degree 1, 2, 3

MSE = np.zeros(3)

for idx, degree in enumerate(range(1, 4)):
    print(idx, degree)
    MSE[idx] = evalMSE(Auto_train, Auto_valid, degree) 

MSE

Try a different training, and validation split by setting a different random_state

The mses are different

In [ ]:
Auto_train, Auto_valid = # TODO:  repeat the above fitting for a different random state

MSE = np.zeros(3)
# TODO: repeat the above fitting for a different random state
#
# TODO

MSE

# An alternative implementation of polynomial features, use PolynomialFeatures

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
Auto_train[['horsepower']].shape

In [ ]:
# evalMSE() takes a model string as well as a training and test set and returns the MSE on the test set

def evalMSE(train, test, d):
    poly_features = PolynomialFeatures(degree=d)
    X_train = poly_features.fit_transform(train[['horsepower']]) 
    y_train = train['mpg']
    
    X_test = poly_features.fit_transform(test[['horsepower']])
    y_test = test['mpg']
    
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [ ]:
Auto_train, Auto_valid = # TODO
MSE = np.zeros(3)
# TODO: fit polynomial of degree 1, 2, 3
# TODO: use evalMSE to get the MSE for each polynomial

MSE

### Cross-validation

LOOCV for Ordinary Least Squares

In [ ]:
# TODO: use cross_validate to get the LOOCV error of OLS
# similar to above, map 'horsepower' to 'mpg' with a polynomial using cross_validate
M = sklearn_sm(sm.OLS)
H = np.array(Auto['horsepower'])
X = sm.add_constant(H)
Y = Auto['mpg']
cv_results = # TODO: implement CV
cv_err = # TODO: get mean CV score
cv_err

An alternative implementation of LOOCV without adding a constant term

In [ ]:
# an intercept term is included
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
## cv = Auto.shape[0] is LOOCV
cv_results = cross_validate(hp_model, X, Y, cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

LOOCV for polynomial regressions

define the polynomials of horsepower using np.power.outer

In [ ]:
# TODO: implement polynomial regression and use cross_validate to get the LOOCV of each polynomial from degree 1, 2, 3, 4, and 5

cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
# if terms are not specified in sklean_sm, then sm.OLS uses all the columns in X without including an intercept term
M = sklearn_sm(sm.OLS)
# TODO for i, d in enumerate(range(1,6)):
#     
# 
#    
# TODO
cv_error

K-fold cross validation

In [ ]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])

cv = KFold(n_splits=10, shuffle=True, random_state=0) # use same splits for each degree
# TODO: repeat the above but now with k-fold CV
for i, d in enumerate(range(1,6)):
    # TODO
    # TODO
    # TODO
    
cv_error

### Implement bootstrap

Estimating the standard deviation of a statistic of Interest

In [ ]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

alpha_func(Portfolio, range(100))

In [ ]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100, 100, replace=True))

In [ ]:
# TODO: use bootstrap for B times and compute the standard deviation of alpha

def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    # TODO for _ in range(B):
    #    idx = TODO
    #    value = func(D, idx)
    #    first_ += value
    #    second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000, seed=0)
alpha_SE

Estimating the standard deviation of a linear regression model coefficients

In [ ]:
print( Auto['horsepower'].shape )

print( Auto['mpg'].shape )

n = 392

idx = rng.choice(n, n, replace=True)

print(idx)

A = Auto['mpg']

A[idx]

In [ ]:
# TODO: use bootstrap to get the standard deviation of the linear model coefficients

def boot_OLS(X, Y, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    first_, second_ = np.zeros((2,)), np.zeros((2,))
    
    for _ in range(B):
        idx = rng.choice(n, n, replace=True)
        #print(idx)
        # TODO
        # 
        # add constant
        # 
        # sm.OLS does not automatically include an intercept term
        # TODO
        #print(value)
        first_ += value
        second_ += value * value

    #print(first_, second_)
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
# print the standard deviation of model coefficients
boot_OLS(Auto['horsepower'], Auto['mpg'])

Estimating the standard deviation of polynomial regression coefficients

In [ ]:
# TODO: use bootstrap to compute the standard deviations of the model coefficients for polynomial regression

def boot_polynomial_regression(X, Y, d, B=1000, seed=0):
    # polynomial degree is d
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    first_, second_ = np.zeros((d+1,)), np.zeros((d+1,))
    
    org_Y = Y.copy()
    org_X = X.copy()
    org_X = # TODO: transform oreg_X to powers of org_X using np.power.outer
    print(sm.OLS(org_Y, org_X).fit().params)
    
    for _ in range(B):
        idx = rng.choice(n, n, replace=True)
        Y_ = # TODO
        X_ = # TODO
        # construct polynomials of X
        X_ = # TODO
        value = # TODO
        first_ += value
        second_ += value * value
        
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
# polynomial degree is 1
boot_polynomial_regression(Auto['horsepower'], Auto['mpg'], 1)

In [ ]:
# polynomial degree is 2
boot_polynomial_regression(Auto['horsepower'], Auto['mpg'], 2)

Use bootstrap to compute the mean of a statistics

In [ ]:
# TODO: Use Bootstrap to get the standard deviation of the mean

def boot_mean(X, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = X.shape[0]
    
    print("mean", np.mean(X))
    
    for _ in range(B):
        idx = # TODO
        value = # TODO
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [ ]:
# standard deviation of 'mpg'
boot_mean(Auto['mpg'])

In [ ]:
# standard deviation of 'horsepower'
boot_mean(Auto['horsepower'])

# Optional module: An alternative implementation

In [ ]:
def boot_OLS(model_matrix, response, D, idx):
    #print('D', D)
    #print('idx')
    print(idx)
    D_ = D.iloc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [ ]:
from functools import partial
from sklearn.base import clone

In [ ]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [ ]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto, rng.choice(392, 392, replace=True)) for _ in range(10)])

In [ ]:
def boot_OLS(model_matrix, response, D, idx):
    #print('D', D)
    #print('idx')
    print(idx)
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [ ]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
hp_se = boot_SE(hp_func, Auto, B=1000, seed=10)
hp_se

In [ ]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

In [ ]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, Auto, B=1000)

In [ ]:
M = sm.OLS(Auto['mpg'], quad_model.fit_transform(Auto))
summarize(M.fit())['std err']